In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import glob
os.getcwd()
pd.set_option('display.max_columns', 70)

In [ ]:
#For commercial stop
### PLEASE CHANGE THE FOLDER PATH
non_comstop = pd.read_parquet("/Volumes/Tuyen Nguyen (Henry)/Downloads/tjj_allptcars")
non_comstop = non_comstop[(non_comstop["REF_REPERAGE_NO"] > 0) & (non_comstop["COMMERCIAL_STOP_NATURE"] != 0)] #Filter for all comercial stops
hours_range = [5, 6, 7, 8, 9, 10, 15, 16, 17, 18, 19, 20]
non_comstop = non_comstop[non_comstop['THR_ARR'].dt.hour.isin(hours_range)]
non_comstop['TRAIN_DEPARTURE_DATE'] = pd.to_datetime(non_comstop['TRAIN_DEPARTURE_DATE'], format='%Y-%m-%d')
non_comstop

In [ ]:
PTCAR = pd.read_csv("./data/PTCAR.csv",sep=";")
non_comstop_ptcar = non_comstop.merge(PTCAR,on='REF_PTCAR_NO',how = 'left')
non_comstop_ptcar = non_comstop_ptcar[non_comstop_ptcar['LONG_NAME_NL'].notna()]
non_comstop_ptcar = non_comstop_ptcar[non_comstop_ptcar['THR_ARR'] != '1900-01-01 00:00:00'].reset_index()
#Any trains arrive within HR_ARR_3min and HR_ARR are behind the the target train
non_comstop_ptcar['THR_ARR_3min'] = non_comstop_ptcar['THR_ARR'] + timedelta(minutes=3)
non_comstop_ptcar = non_comstop_ptcar[['REF_TOD_TRAIN_ID','THR_ARR','THR_ARR_3min','LONG_NAME_NL','TRAIN_DEPARTURE_DATE']]
non_comstop_ptcar = non_comstop_ptcar.reset_index()
non_comstop_ptcar = non_comstop_ptcar.drop(columns='index')
non_comstop_ptcar

In [ ]:
### PLEASE CHANGE THE FOLDER PATH
folder_path = '/Volumes/Tuyen Nguyen (Henry)/Traindynamic'
non_comstop_ptcar = non_comstop_ptcar.sort_values(by = 'REF_TOD_TRAIN_ID')
unique_dates = non_comstop_ptcar['TRAIN_DEPARTURE_DATE'].unique()
grouped_data = []

for date in unique_dates:
    data_date = non_comstop_ptcar[non_comstop_ptcar['TRAIN_DEPARTURE_DATE'] == date]
    grouped = data_date.groupby(['LONG_NAME_NL'])
    for name, group in grouped:
        group = group.drop_duplicates(subset=['REF_TOD_TRAIN_ID','LONG_NAME_NL'],keep = 'last')
        ref_ids = group['REF_TOD_TRAIN_ID'].values
        mask = (ref_ids[:, np.newaxis] != ref_ids) & \
               (group['THR_ARR'].values[:, np.newaxis] >= group['THR_ARR'].values) & \
               (group['THR_ARR'].values[:, np.newaxis] <= group['THR_ARR_3min'].values)
        ohe = pd.get_dummies(group['REF_TOD_TRAIN_ID'])
        np.fill_diagonal(ohe.values, 0)
        group_ohe = pd.concat([group, ohe], axis=1)
        #Replace the mask with the corresponding train ID
        group_ohe.iloc[:, 5:] = np.where(mask, 1, group_ohe.iloc[:, 5:])
        
        # Create the filename based on the day
        day = np.datetime_as_string(date, unit='D')
        name_with_underscores = name.replace(" ", "-").replace('/', '').replace(':', '')
        filename = f"{day}_{name_with_underscores}.csv"  # Using 'LONG_NAME_NL' and day in the filename
        #Save the group dataframe as a CSV file in the specified folder
        group_ohe.to_csv(os.path.join(folder_path, filename), index=False)